# CellMap for human bone marrow cell data

The analysis example using human bone marrow cell data ([Setty et al. (2019)](https://doi.org/10.1242/dev.173849)). 

[RECODE](https://github.com/yusuke-imoto-lab/RECODE) and [UNitVelo](https://ui.adsabs.harvard.edu/link_gateway/2022NatCo..13.6586G/doi:10.1038/s41467-022-34188-7) are used for the noise reduction and velocity inference, respectively.

## Regular procedure

In [1]:
import cellmap
import anndata
import matplotlib.pyplot as plt
import numpy as np
import scanpy

The dataset is installed using the [scvelo](https://scvelo.readthedocs.io/en/stable/) package. 

In [9]:
import scvelo as scv
adata = scv.datasets.bonemarrow()
adata

AnnData object with n_obs × n_vars = 5780 × 14319
    obs: 'clusters', 'palantir_pseudotime'
    uns: 'clusters_colors'
    obsm: 'X_tsne'
    layers: 'spliced', 'unspliced'

Noise reduction by [RECODE](https://github.com/yusuke-imoto-lab/RECODE).

In [ ]:
import screcode
recode = screcode.RECODE()
adata = recode.fit_transform(adata)
adata.obsm['RECODE_log'] = scanpy.pp.log1p(adata.obsm['RECODE'])
recode.report()

start RECODE for scRNA-seq


array([[0.0000000000e+00, 1.7112445579e-03, 0.0000000000e+00, ...,
        0.0000000000e+00, 7.5232679307e-01, 1.2980218535e-01],
       [4.0880335843e-02, 0.0000000000e+00, 0.0000000000e+00, ...,
        0.0000000000e+00, 3.0485631173e-01, 9.2875758302e-02],
       [2.0096601080e-03, 4.3938891389e-04, 0.0000000000e+00, ...,
        0.0000000000e+00, 2.2962659907e-01, 0.0000000000e+00],
       ...,
       [4.4000732537e-02, 0.0000000000e+00, 0.0000000000e+00, ...,
        0.0000000000e+00, 8.5532921447e-01, 0.0000000000e+00],
       [4.0377235170e-02, 0.0000000000e+00, 0.0000000000e+00, ...,
        0.0000000000e+00, 1.1819125543e+00, 1.1344619398e-01],
       [2.9025203357e-02, 2.9840457235e-04, 0.0000000000e+00, ...,
        0.0000000000e+00, 8.3496159266e-01, 3.9188305329e-01]])

Compute velocity by UnitVelo and show the stream on UMAP. 

In [6]:
import unitvelo as utv

velo_config = utv.config.Configuration()
velo_config.VGENES = 'offset'
velo_config.R2_ADJUST = False
velo_config.IROOT = 'HSC_1'
velo_config.FIT_OPTION = '1'
adata = utv.run_model(adata,label = 'clusters', config_file=velo_config)

------> Manully Specified Parameters <------
R2_ADJUST:	False
VGENES:	offset
IROOT:	HSC_1
------> Model Configuration Settings <------
N_TOP_GENES:	2000
LEARNING_RATE:	0.01
FIT_OPTION:	1
DENSITY:	SVD
REORDER_CELL:	Soft_Reorder
AGGREGATE_T:	True
GENE_PRIOR:	None
--------------------------------------------

Current working dir is /mnt/d/GitHub/CellMap/tutorial.
Results will be stored in res folder
Filtered out 7837 genes that are detected 20 counts (shared).
Normalized count data: X, spliced, unspliced.
Extracted 2000 highly variable genes.
Logarithmized X.
Extracted 2000 highly variable genes.
Computing moments for 2000 genes with n_neighbors: 30 and n_pcs: 30
computing neighbors
    finished (0:00:07) --> added 
    'distances' and 'connectivities', weighted adjacency matrices (adata.obsp)
computing moments based on connectivities
    finished (0:00:00) --> added 
    'Ms' and 'Mu', moments of un/spliced abundances (adata.layers)

# of velocity genes 1382 (Criterion: positive regressi

Loss (Total): 4508.688, (Spliced): 2273.289, (Unspliced): 2235.399:  70%|█████▌  | 8435/12000 [1:02:15<26:18,  2.26it/s]

KeyboardInterrupt



In [ ]:
scv.pl.velocity_embedding_stream(adata)

In [ ]:
%%time
import cellmap

cellmap.Hodge_decomposition(adata, exp_2d_key='X_tsne', vel_2d_key='velocity_tsne', alpha=0.1)

In [ ]:
cellmap.view(adata, basis='X_tsne', show_graph = True, cluster_key='clusters', cmap='jet')

In [ ]:
cellmap.view_cluster(adata, basis='X_tsne', show_graph=True, cluster_key='clusters', s=50,cmap='jet')

In [ ]:
cellmap.view_surface(adata, basis='X_tsne', cluster_key ='clusters',cmap='jet')

In [ ]:
cellmap.view_surface_3D(adata, basis='X_tsne', cluster_key ='clusters', elev=50, azim=65)

In [ ]:
cellmap.view_surface_3D_cluster(adata, basis='X_tsne', cluster_key ='clusters',elev=50,azim=65,s=30);

### Write expression and potential data as CSV file for **[CellMap viewer](https://github.com/yusuke-imoto-lab/CellMapViewer)**. 

In [ ]:
cellmap.write(adata, basis='X_tsne', filename='CellMap_tutorial_hippocampus',expression_key='RECODE_log')

## Changing parameter $\alpha$

The parameter $\alpha$ adjust the rate of original/reduced dimensional information of gene expression and velocity. 

$\alpha=0$ uses only two-dimensional inormation; in contrast, $\alpha=1$ uses the original (non-dimensionaly reduction) information. 

In [ ]:
alpha_set = [0,0.5,1]
for alpha in alpha_set:
    cellmap.Hodge_decomposition(adata, exp_2d_key='X_tsne', vel_2d_key='velocity_tsne',potential_key='Hodge_potential_%0.1f' % alpha,alpha=alpha)
adata

In [ ]:
for alpha in alpha_set:
    cellmap.view(adata, basis='X_tsne', show_graph = True, potential_key='Hodge_potential_%0.1f' % alpha,cluster_key='clusters',title='alpha=%0.1f' % alpha,s=1)

In [ ]:
for alpha in alpha_set:
    cellmap.view_surface_3D(adata, basis='X_tsne', cluster_key ='clusters',potential_key='Hodge_potential_%0.1f' % alpha,elev=50,azim=65,title='alpha=%0.1f' % alpha)